# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import requests
import json
import pprint
import gmaps
import gmaps.datasets

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
#import csv into a dataframe
import_data="../output_data/hwfile1.csv"
data_file_pd = pd.read_csv(import_data)

#show what the data frame looks like 
data_file_pd.set_index(["City_ID"],inplace=False)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City_ID,,,,,,,,,
0,Yellowknife,20,CA,1584300276,63,62.46,-114.35,26.60,18.34
1,Itarema,57,BR,1584300010,81,-2.92,-39.92,83.64,6.38
2,Odienné,97,CI,1584300276,18,9.51,-7.57,93.72,3.00
3,Leshukonskoye,74,RU,1584300276,84,64.90,45.76,19.56,11.21
4,Cayenne,75,GF,1584300276,62,4.93,-52.33,86.00,11.41
...,...,...,...,...,...,...,...,...,...
495,Kindersley,90,CA,1584300316,72,51.47,-109.17,10.40,9.17
496,Zhiryatino,77,RU,1584300316,51,53.22,33.73,28.92,12.84
497,Albany,1,US,1584300316,22,42.60,-73.97,46.99,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = data_file_pd[["Lat", "Lng"]]
humidity=data_file_pd["Humidity"].astype(float)

#Configure gmaps.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=data_file_pd["Humidity"].max(),
                                 point_radius = 1)
#Add Heatmap layer to map.
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Narrow down the cities to fit weather conditions.
vacation_data=data_file_pd.loc[(data_file_pd["Max Temp"]<=80)
                               &(data_file_pd["Max Temp"]>=70)
                               &(data_file_pd["Wind Speed"]<10)
                                &(data_file_pd["Cloudiness"]==0),:]

#Drop any rows will null values.
vacation_data.reset_index(drop=True,inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Store into variable named hotel_df.
hotel_df=vacation_data

#Add a "Hotel Name" and "adress" column to the DataFrame.
hotel_df["Hotel Name"]=""
hotel_df['address'] = ""

#show what the dataframe look like
hotel_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,address
0,48,Necochea,0,AR,1584300280,64,-38.55,-58.74,70.00,5.01,,
1,69,Bowen,0,AU,1584300222,93,-20.02,148.23,72.00,1.12,,
2,99,Umluj,0,SA,1584300043,55,25.02,37.27,70.14,7.90,,
3,246,Mendoza,0,AR,1584300174,21,-32.89,-68.83,78.80,6.93,,
4,248,Kasongo,0,CD,1584300295,86,-4.45,26.67,75.00,1.54,,


In [17]:
#create the params for the search 
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
target_search = "Hotel"
target_radius = 5000
target_type="Hotel"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key,
}

#create loop to search for all of the hotels
for index, row in hotel_df.iterrows():
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #get location for hotel
    target_coordinates=f'{row["Lat"]},{row["Lng"]}'
    
    #add the target locations in the parameters
    params["location"] =target_coordinates
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {target_coordinates}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    try:
        print(f"Closest hotel is named: {results[1]['name']}.")
        print(f"Located at: {results[1]['vicinity']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        hotel_df.loc[index, 'address'] = results[1]['vicinity']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: -38.55,-58.74.
Closest hotel is named: dyd -mar Hotel.
Located at: Calle 77 314, Necochea.
------------
Retrieving Results for Index 1: -20.02,148.23.
Closest hotel is named: Castle Motor Lodge Motel.
Located at: 6 Don Street, Bowen.
------------
Retrieving Results for Index 2: 25.02,37.27.
Closest hotel is named: Tanal Hotel Apartment.
Located at: Al Khubar, Umluj.
------------
Retrieving Results for Index 3: -32.89,-68.83.
Closest hotel is named: Hotel Mendoza.
Located at: Avenida España 1210, Mendoza.
------------
Retrieving Results for Index 4: -4.45,26.67.
Closest hotel is named: Cathédrale De Kasongo.
Located at: Congo - Kinshasa.
------------
Retrieving Results for Index 5: -41.15,-71.31.
Closest hotel is named: Hotel Patagonia.
Located at: Avenida Exequiel Bustillo 1231, San Carlos de Bariloche.
------------
Retrieving Results for Index 6: 6.12,81.12.
Closest hotel is named: Bungalow 63.
Located at: No 63 Barrak Street, Hambantota.
------------
R

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))